In [ ]:
import os
from typing import Optional

# External Dependencies:
import boto3
from botocore.config import Config


def get_bedrock_client(
    assumed_role: Optional[str] = None,
    region: Optional[str] = None,
    runtime: Optional[bool] = True,
):
  
    if region is None:
        target_region = os.environ.get("AWS_REGION", os.environ.get("AWS_DEFAULT_REGION"))
    else:
        target_region = region

    print(f"Create new client\n  Using region: {target_region}")
    session_kwargs = {"region_name": target_region}
    client_kwargs = {**session_kwargs}

    profile_name = os.environ.get("AWS_PROFILE")
    if profile_name:
        print(f"  Using profile: {profile_name}")
        session_kwargs["profile_name"] = profile_name

    retry_config = Config(
        region_name=target_region,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
    session = boto3.Session(**session_kwargs)

    if assumed_role:
        print(f"  Using role: {assumed_role}", end='')
        sts = session.client("sts")
        response = sts.assume_role(
            RoleArn=str(assumed_role),
            RoleSessionName="langchain-llm-1"
        )
        print(" ... successful!")
        client_kwargs["aws_access_key_id"] = response["Credentials"]["AccessKeyId"]
        client_kwargs["aws_secret_access_key"] = response["Credentials"]["SecretAccessKey"]
        client_kwargs["aws_session_token"] = response["Credentials"]["SessionToken"]
        

    if runtime:
        service_name='bedrock-runtime'
    else:
        service_name='bedrock'

    client_kwargs["aws_access_key_id"] = os.environ.get("AWS_ACCESS_KEY_ID","")
    client_kwargs["aws_secret_access_key"] = os.environ.get("AWS_SECRET_ACCESS_KEY","")
    
    bedrock_client = session.client(
        service_name=service_name,
        config=retry_config,
        **client_kwargs
    )

    print("boto3 Bedrock client successfully created!")
    print(bedrock_client._endpoint)
    return bedrock_client



## for aksk bedrock
def get_bedrock_aksk(secret_name='chatbot_bedrock', region_name = "us-west-2"):
    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        # For a list of exceptions thrown, see
        # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
        raise e

    # Decrypts secret using the associated KMS key.
    secret = json.loads(get_secret_value_response['SecretString'])
    return secret['BEDROCK_ACCESS_KEY'],secret['BEDROCK_SECRET_KEY']

ACCESS_KEY, SECRET_KEY=get_bedrock_aksk()

#role based initial client#######
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"  # E.g. "us-east-1"
os.environ["AWS_PROFILE"] = "default"
#os.environ["BEDROCK_ASSUME_ROLE"] = "arn:aws:iam::687912291502:role/service-role/AmazonSageMaker-ExecutionRole-20211013T113123"  # E.g. "arn:aws:..."
os.environ["AWS_ACCESS_KEY_ID"]=ACCESS_KEY
os.environ["AWS_SECRET_ACCESS_KEY"]=SECRET_KEY


#新boto3 sdk只能session方式初始化bedrock
boto3_bedrock = get_bedrock_client(
    #assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

parameters_bedrock = {
    "max_tokens_to_sample": 2048,
    #"temperature": 0.5,
    "temperature": 0,
    #"top_k": 250,
    #"top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

bedrock_llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs=parameters_bedrock)
###test the bedrock langchain integration###
#bedrock_llm.predict("Human:how do you describe LLM?\n"+
#           "Assistant:")

bedrock_llm_additional = BedrockModelWrapper(model_id="anthropic.claude-v2", 
                                          client=boto3_bedrock, 
                                          model_kwargs=parameters_bedrock)
bedrock_chain = LLMChain(llm=bedrock_llm_additional, verbose=True)



In [ ]:
import requests
import json
def request_api(language, code):
    data = {
        "language": language, 
        "code": code
    }

    response = requests.post("https://localhost:2000", json=data)
    result = response.json()
    return result

search_tool = Tool(
    name="search_enterprise_documents",
    func=request_api
    description="useful for when you need to searches and returns documents regarding the user's question",
)
codegen_tool = Tool(
    func=bedrock_chain.run,
    name="code generation",
    description="useful for when you need to generate aws cli scripte code",
)
custom_tool_list = [codegen_tool,search_tool]

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

memory = ConversationBufferWindowMemory(k=2)
system_message = SystemMessage(
    content=(
        "You are an experienced AWS CLI expert who can effectively troubleshoot and resolve issues."
        "Generate AWS CLI scripts to troubleshoot user's questions, and use runtime tools to execute the scripts and get result."
        "Ensure the script is clear, concise, and includes specific details about the expected user's question . Specify the length and format for any troubleshooting documentation that needs to be provided."
        "### Note: Please only respond if you have extensive experience with the AWS CLI and can confidently troubleshoot and resolve issues."
    )
)
prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=memory_key)],
)

In [ ]:
agent_executor = initialize_agent(custom_tool_list, bedrock_llm_additional, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, 
                                  verbose=True,max_iterations=3,
                                  handle_parsing_errors=True,
                                  memory=memory,
                                  return_intermediate_steps=True)
agent_executor.agent.llm_chain.prompt.template=prompt
agent_executor.run("没有打tag的aws EC2实例有哪些？")